In [519]:
import tensorflow as tf
import tensorflow_datasets as tfds
# from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, GlobalMaxPooling2D, Activation, Rescaling
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential as SequentialModel
import keras_tuner as kt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools

import os
import time

from pathlib import Path

# tf.random.set_seed(5263)
tf.__version__

'2.14.0'

In [520]:
# Set up datasets before passing to the model


# TODO: Image shape should be taken from the dataset, not hardcoded
image_shape = (400, 400, 3)

# Normalise image values to 0-1 from 0-255
img_normaliser = layers.Rescaling(1./255)

In [521]:
incomplete_dataset_path = Path("") / "datasets" / "data" / "TEST-okinimesumama-expert-24-played-incomplete"

df = pd.read_csv(incomplete_dataset_path / "labels.csv")

df.head()

,file_name,b1,b2,b3,b4,b5,b6,b7,b8
0,okinimesumama-expert-24-played-incomplete-0000...,False,False,False,False,False,False,False,False
1,okinimesumama-expert-24-played-incomplete-0000...,False,False,False,False,False,False,False,False
2,okinimesumama-expert-24-played-incomplete-0000...,False,False,False,False,False,False,False,False
3,okinimesumama-expert-24-played-incomplete-0000...,False,False,False,False,False,False,False,False
4,okinimesumama-expert-24-played-incomplete-0000...,False,False,False,False,False,False,False,False


In [522]:
file_names = df["file_name"].values
for count, i in enumerate(file_names):
    file_names[count] = str(incomplete_dataset_path / i)
file_names[:100]  # Truncated to stop the notebook from becoming huge

array(['datasets\\data\\TEST-okinimesumama-expert-24-played-incomplete\\okinimesumama-expert-24-played-incomplete-0000000000.png',
       'datasets\\data\\TEST-okinimesumama-expert-24-played-incomplete\\okinimesumama-expert-24-played-incomplete-0000000001.png',
       'datasets\\data\\TEST-okinimesumama-expert-24-played-incomplete\\okinimesumama-expert-24-played-incomplete-0000000002.png',
       'datasets\\data\\TEST-okinimesumama-expert-24-played-incomplete\\okinimesumama-expert-24-played-incomplete-0000000003.png',
       'datasets\\data\\TEST-okinimesumama-expert-24-played-incomplete\\okinimesumama-expert-24-played-incomplete-0000000004.png',
       'datasets\\data\\TEST-okinimesumama-expert-24-played-incomplete\\okinimesumama-expert-24-played-incomplete-0000000005.png',
       'datasets\\data\\TEST-okinimesumama-expert-24-played-incomplete\\okinimesumama-expert-24-played-incomplete-0000000006.png',
       'datasets\\data\\TEST-okinimesumama-expert-24-played-incomplete\\okinimesuma

In [523]:
# Make a copy for labels in case we need everything together later - `df` is intact
labels = df.drop("file_name", axis=1)
labels.head()

,b1,b2,b3,b4,b5,b6,b7,b8
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False


In [524]:
# Ensure format of labels is correct
dataset_train = tf.data.Dataset.from_tensor_slices((file_names, labels))
dataset_train

<_TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(8,), dtype=tf.bool, name=None))>

In [525]:
label_list = list(labels.to_numpy())
for count, label in enumerate(label_list):
    label_list[count] = tuple(label)

label_list

[(False, False, False, False, False, False, False, False),
 (False, False, False, False, False, False, False, False),
 (False, False, False, False, False, False, False, False),
 (False, False, False, False, False, False, False, False),
 (False, False, False, False, False, False, False, False),
 (False, False, False, False, False, False, False, False),
 (False, False, False, False, False, False, False, False),
 (False, False, False, False, False, False, False, False),
 (False, False, False, False, False, False, False, False),
 (False, False, False, False, False, False, False, False),
 (False, False, False, False, False, False, False, False),
 (False, False, False, False, False, False, False, False),
 (False, False, False, False, False, False, False, False),
 (False, False, False, False, False, False, False, False),
 (False, False, False, False, False, False, False, False),
 (False, False, False, False, False, False, False, False),
 (False, False, False, False, False, False, False, False

In [526]:
string_base_path = str(incomplete_dataset_path)
def read_image(image_path, labels):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = img_normaliser(image)
    image.set_shape([400, 400, 3])
    image = tf.expand_dims(image, 0)
    labels = [labels]
    
    return image, labels

dataset_train = dataset_train.map(read_image)
dataset_train

<_MapDataset element_spec=(TensorSpec(shape=(1, 400, 400, 3), dtype=tf.float32, name=None), TensorSpec(shape=(1, 8), dtype=tf.bool, name=None))>

In [527]:
# # TODO: Look into moving labels to directory structure instead of csv file, then infer labels via keras
# keras_dataset = tf.keras.utils.image_dataset_from_directory(
#     str(incomplete_dataset_path),
#     label_list,
#     label_mode="categorical",
#     image_size=(400, 400)
# )
# keras_dataset

In [528]:
# dataset_path = Path("") / "datasets" / "data" / "TEST-okinimesumama-expert-24-played-incomplete_ordered_1699049249.49"
# 
# folder_name = "b1_b2_b3_b4"
# image_path = dataset_path / folder_name
# print(image_path)
# 
# img_path = image_path / os.listdir(image_path)[0]
# img = tf.image.decode_png(tf.io.read_file(str(img_path)))
# label_list = img_path.parts[-2]
# label_list
# labels = label_list.split("_")
# 
# label_arr = ["b1" in labels, "b2" in labels, "b3" in labels, "b4" in labels, 
#              "b5" in labels, "b6" in labels, "b7" in labels, "b8" in labels]
# label_arr

In [529]:
# def load_image(image_name, image_labels):
#     # print(image_name)
#     image_path = "datassets/data/TEST-okinimesumama-expert-24-played-incomplete/" + image_name
#     image = tf.io.read_file(image_path)
#     image = tf.image.decode_png(image)
#     # print(image.)
#     rescaler = layers.Rescaling(1./255)
#     rescaled_image = rescaler(image)
#     return image, image_labels
# 
# def augment_image(image, image_labels):
#     # TODO: Add applicable augmentation (brightness, saturation?, slight scaling?)
#     return image, image_labels
# 
# # The top line of the image is all black due to the screen outputting a circle for the play area, so the output here is not alarming (use indexing to select a specific pixel to prove that the file is correctly read)
# # load_image(file_names[0], labels.head(1))
# 
# dataset_train = dataset_train.map(load_image)
# dataset_train[0]

In [530]:
wamai_model = SequentialModel([
    
    # layers.InputLayer(input_shape=(400, 400, 3)),
    layers.Conv2D(filters = 32, kernel_size = (3,3), strides = 2, input_shape = image_shape),
    layers.Activation('relu'),
    layers.BatchNormalization(),

    layers.Conv2D(filters = 64, kernel_size = (3,3), strides = 2),
    layers.Activation('relu'),
    layers.BatchNormalization(),

    layers.Conv2D(filters = 128, kernel_size = (3,3), strides = 2),
    layers.Activation('relu'),
    layers.BatchNormalization(),

    layers.MaxPool2D(pool_size = (2, 2)),
    layers.Conv2D(filters = 32, kernel_size = (3,3)),
    layers.Activation('relu'),
    layers.BatchNormalization(),

    layers.MaxPool2D(pool_size = (2, 2)),
    layers.Flatten(),

    layers.Dense(units = 512),
    layers.Activation('relu'),

    layers.Dense(units = 512),
    layers.Activation('relu'),

    layers.Dense(units = 8),
    layers.Activation('softmax')

])

wamai_model.summary()

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_96 (Conv2D)          (None, 199, 199, 32)      896       
                                                                 
 activation_168 (Activation  (None, 199, 199, 32)      0         
 )                                                               
                                                                 
 batch_normalization_96 (Ba  (None, 199, 199, 32)      128       
 tchNormalization)                                               
                                                                 
 conv2d_97 (Conv2D)          (None, 99, 99, 64)        18496     
                                                                 
 activation_169 (Activation  (None, 99, 99, 64)        0         
 )                                                               
                                                     

In [531]:
wamai_model.compile(optimizer=tf.keras.optimizers.Adam(),
                    loss='binary_crossentropy',  # Binary crossentropy predicts a probability for each label - much more like the outputs wanted
                    metrics=["accuracy"])

In [532]:
model_file_path = Path("") / "models" / "WA.M.AI proto 1" / f"WA.M.AI-p-1_binary_high-epoch{time.time()}.ckpt"
model_file_path_string = str(model_file_path)
model_folder = model_file_path.parent
if not os.path.exists(model_folder):
    os.makedirs(model_folder)

checkpoint_filepath = model_file_path.parent / "WA.M.AI-p-binary-epoch{epoch:02d}-{val_loss:.2f}.hdf5"

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

wamai_model.fit(dataset_train, epochs=100)

# wamai_model.save_weights(model_file_path_string)

Epoch 1/2
2260/2260 [==============================] - 184s 80ms/step - loss: 0.5601 - accuracy: 0.1996
Epoch 2/2
2260/2260 [==============================] - 177s 78ms/step - loss: 0.5486 - accuracy: 0.1854


In [533]:
dataset_test = tf.data.Dataset.from_tensor_slices((["datasets\\data\\TEST-okinimesumama-expert-24-played-incomplete\\okinimesumama-expert-24-played-incomplete-0000001057.png", "datasets\\data\\TEST-okinimesumama-expert-24-played-incomplete\\okinimesumama-expert-24-played-incomplete-0000001058.png"], [[False,False,True,True,False,False,False,False], [False,False,True,True,False,False,False,False]]))
# dataset_test
dataset_test = dataset_test.map(read_image)
dataset_test

<_MapDataset element_spec=(TensorSpec(shape=(1, 400, 400, 3), dtype=tf.float32, name=None), TensorSpec(shape=(1, 8), dtype=tf.bool, name=None))>

In [534]:
wamai_model.predict(dataset_test)

2/2 [==============================] - 0s 18ms/step


array([[0.13152727, 0.16238569, 0.11336713, 0.12845396, 0.10092722,
        0.13302656, 0.1305018 , 0.09981041],
       [0.13152727, 0.16238569, 0.11336713, 0.12845396, 0.10092722,
        0.13302656, 0.1305018 , 0.09981041]], dtype=float32)